In [1]:
!pip install fastapi Pillow python-multipart torch transformers uvicorn nest-asyncio pyngrok

## External authentication step Added

In [2]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()  # loads variables from .env
login(token=os.getenv("HF_TOKEN"))


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
! pip install openai

In [4]:
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


In [5]:
!curl -I https://huggingface.co


HTTP/2 200 
content-type: text/html; charset=utf-8
content-length: 131802
date: Fri, 27 Jun 2025 02:25:39 GMT
x-powered-by: huggingface-moon
cross-origin-opener-policy: same-origin
referrer-policy: strict-origin-when-cross-origin
x-request-id: Root=1-685e0123-551e6d770b34e1594e949628
x-frame-options: DENY
etag: W/"202da-Jq2KpbDUtnEhJm4cmyVkgVXX9oE"
x-cache: Hit from cloudfront
via: 1.1 43cd35d154fe606336f72858d8bd76ec.cloudfront.net (CloudFront)
x-amz-cf-pop: MSP50-P1
x-amz-cf-id: Uf5UTfdoJq1O_lawccTMs9qVy_bxXkEcUAdm7nNOrAcXoltAp3flNQ==
age: 46



In [6]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")



In [7]:
print("Model loaded:", type(model))


Model loaded: <class 'transformers.models.vilt.modeling_vilt.ViltForQuestionAnswering'>


In [8]:
from huggingface_hub import whoami
whoami()


{'type': 'user',
 'id': '685dddd526e41cf57bbfc12c',
 'name': 'AshleY520',
 'fullname': 'Yiyi Sun',
 'isPro': False,
 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/0n9drzcocDbNxfRd5hyQe.png',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'ML Optimization',
   'role': 'fineGrained',
   'createdAt': '2025-06-26T23:56:19.674Z',
   'fineGrained': {'canReadGatedRepos': False,
    'global': [],
    'scoped': [{'entity': {'_id': '685dddd526e41cf57bbfc12c',
       'type': 'user',
       'name': 'AshleY520'},
      'permissions': []}]}}}}

In [14]:
# prepare image + question
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

text = "What are the colors of the cats?"

# prepare inputs
encoding = processor(image, text, return_tensors="pt")


In [15]:
def vqa_predict(text, image):
    """
    Performs Visual Question Answering using the VILT model.
    
    Args:
        text (str): The question.
        image (PIL.Image): The input image.

    Returns:
        str: The predicted answer.
    """
    # Encode image and text
    encoding = processor(image, text, return_tensors="pt")

    # Forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    
    return model.config.id2label[idx]


In [ ]:

# # forward pass
# outputs = model(**encoding)
# logits = outputs.logits
# idx = logits.argmax(-1).item()


# print("Predicted answer:", model.config.id2label[idx])

# # TODO: put above code into a function that accepts image and text as input

Predicted answer: brown and black


In [16]:
def model_pipeline(text: str, image: Image):
    # prepare inputs
    encoding = processor(image, text, return_tensors="pt")

    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()

    return  model.config.id2label[idx]

In [17]:
from fastapi import FastAPI, UploadFile, File, Form
import io
from PIL import Image

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/ask")
async def ask(text: str = Form(...), image: UploadFile = File(...)):
    content = await image.read()

    try:
        image_pil = Image.open(io.BytesIO(content)).convert("RGB")
    except Exception as e:
        return {"error": f"Invalid image file: {str(e)}"}

    answer = vqa_predict(text, image_pil)
    return {"answer": answer}


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2z4XmxyFWZ5WudtkrmuTpM0VD0n_3atxcB76Bix1gVaj3YByN")
tunnel = ngrok.connect(8000)
print("Public URL:", tunnel.public_url)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Public URL: https://afd6-174-82-205-246.ngrok-free.app


t=2025-06-26T21:33:57-0500 lvl=warn msg="failed to open private leg" id=23f42bfdf160 privaddr=localhost:8000 err="dial tcp [::1]:8000: connect: connection refused"
t=2025-06-26T21:33:58-0500 lvl=warn msg="failed to open private leg" id=471e5326ce08 privaddr=localhost:8000 err="dial tcp [::1]:8000: connect: connection refused"
t=2025-06-26T21:34:11-0500 lvl=warn msg="failed to open private leg" id=e65a850502b7 privaddr=localhost:8000 err="dial tcp [::1]:8000: connect: connection refused"
t=2025-06-26T21:34:12-0500 lvl=warn msg="failed to open private leg" id=d87c0c2c1e2a privaddr=localhost:8000 err="dial tcp [::1]:8000: connect: connection refused"
